In [1]:
import os
import sys

os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.1-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [2]:
import pyspark
myConf=pyspark.SparkConf() # SparkSession에 필요한 설정을 삽입해서 만듦. 지금은 공란
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config("spark.sql.warehouse.dir", "C:\users\g312")\
    .getOrCreate()

In [3]:
print spark.version

2.0.0


## DataFrame

In [4]:
myList=[('1','kim, js',170),
        ('1','lee, sm', 175),
        ('2','lim, yg',180),
        ('2','lee',170)]

In [5]:
myDf=spark.createDataFrame(myList) #내부든 외부든 createDataFrame쓰면 DF가 생성됨

In [6]:
myDf.printSchema() #앞 두 column은 string ('')으로, 마지막 column은 long으로 자동 지정
print myDf.take(1) #column이름이 없으므로 열은 _1, _2와 같이 표시됨

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: long (nullable = true)

[Row(_1=u'1', _2=u'kim, js', _3=170)]


In [7]:
print spark.createDataFrame(myList, ['year','name','height']).take(1)
#이와 같이 열의 이름을 설정하면 출력시에도 열의 이름으로 보여준다.

[Row(year=u'1', name=u'kim, js', height=170)]


In [8]:
names = ["kim","lee","lee","lim"]
items = ["espresso","latte","americano","affocato","long black","macciato"]
df = spark.createDataFrame([(names[i%4], items[i%6]) for i in range(100)],\
                           ["name","item"])
#위에 있는 이름과 아이템들을 순서대로 100번 반복반환한다.
#names[i%4], items[i%6] i->0, 1, 2, 3 and i->0, 1, 2, 3, 4, 5
df.show(10)

+----+----------+
|name|      item|
+----+----------+
| kim|  espresso|
| lee|     latte|
| lee| americano|
| lim|  affocato|
| kim|long black|
| lee|  macciato|
| lee|  espresso|
| lim|     latte|
| kim| americano|
| lee|  affocato|
+----+----------+
only showing top 10 rows



In [9]:
df.select(df.item.substr(1, 3).alias("short name")).show(3)
#substr -> 선택할 문자의 range(현재는 세글자), alias -> 컬럼의 이름

+----------+
|short name|
+----------+
|       esp|
|       lat|
|       ame|
+----------+
only showing top 3 rows



In [10]:
from pyspark.sql import Row # Row 객체를 사용한다.
Person = Row('year','name', 'height') #Row안(속성)에는 column의 이름이 들어감.
row1=Person('1','kim, js',170)#Person이란 Row 객체에 맞춰 행을 입력.

In [11]:
print "row1: ", row1.year, row1.name #row.key 방식으로 출력, row[key]로도 출력된다.

row1:  1 kim, js


In [12]:
myRows = [row1, #myRows(DataFrame)에 들어갈 객체로 만들어진 객체를 넣을 수 있다.
          Person('1','lee, sm', 175),
          Person('2','lim, yg',180),
          Person('2','lee',170)] 
#DataFrame을 List의 형태로 생성하는 방식

In [13]:
myDf=spark.createDataFrame(myRows)

In [14]:
print myDf.printSchema()
myDf.show()

root
 |-- year: string (nullable = true)
 |-- name: string (nullable = true)
 |-- height: long (nullable = true)

None
+----+-------+------+
|year|   name|height|
+----+-------+------+
|   1|kim, js|   170|
|   1|lee, sm|   175|
|   2|lim, yg|   180|
|   2|    lee|   170|
+----+-------+------+



In [15]:
#Schema를 정의해보자.
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType
mySchema=StructType([
    StructField("year", StringType(), True),
    StructField("name", StringType(), True),
    StructField("height", IntegerType(), True)
])
#현재 StructField엔 column의 명칭, 타입, 그리고 Null의 허용여부가 순서대로 입력되어 있다


In [16]:
myDf=spark.createDataFrame(myRows, mySchema) #처음과 다르게 리스트와 스키마를 같이 부름.
myDf.printSchema() #결과에서 보이듯이 자동지정된 Long이 아닌 Int형이다.
myDf.take(1) 

root
 |-- year: string (nullable = true)
 |-- name: string (nullable = true)
 |-- height: integer (nullable = true)



[Row(year=u'1', name=u'kim, js', height=170)]

In [17]:
#Rdd로 DF를 생성해보자.
from pyspark.sql import Row

myList=[('1','kim, js',170),('1','lee, sm', 175),('2','lim, yg',180),('2','lee',170)]
myRdd = spark.sparkContext.parallelize(myList)

In [18]:
rddDf=myRdd.toDF() #toDF 함수로 RDD가 DF로 형변환된다. Schema가 없으므로 자동지정됨.
rddDf.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: long (nullable = true)



In [19]:
rddDf=spark.createDataFrame(myRdd) #RDD를 생성함수로도 변환시킬 수 있다.
rddDf.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: long (nullable = true)



In [29]:
rddDf.where(rddDf._3 < 175)\
    .select([rddDf._1, rddDf._2])\
    .show() 
#데이터에서 _3(세번째 column)의 value<175인 row는?
#1, 2번 column을 선택
#선택된 column과 row만 보여준다.

+---+-------+
| _1|     _2|
+---+-------+
|  1|kim, js|
|  2|    lee|
+---+-------+



In [21]:
rddDf.groupby(rddDf._1).max().show()
#이거 어려워 보이는데 쉬움.
#일단 DF를 rddDf._1(key)로 그룹핑하고 그룹마다의 value의 최댓값을 골라 이를 출력한다. 

+---+-------+
| _1|max(_3)|
+---+-------+
|  1|    175|
|  2|    180|
+---+-------+



In [22]:
#Row 객체로 위를 구현해보자.
_myRdd=myRdd.map(lambda x:Row(year=int(x[0]),name=x[1],height=int(x[2])))
#year는 int로 자동 형변환된다.
#맵에서 column이름을 정해준다.
#RDD에서도 Row 객체를 쓸 수 있구나.

In [23]:
_myDf=spark.createDataFrame(_myRdd) # 위와 같은 과정
_myDf.printSchema() #위에선 int로 받았지만, schema 설정한 것은 아니므로 자동지정
_myDf.take(1)

root
 |-- height: long (nullable = true)
 |-- name: string (nullable = true)
 |-- year: long (nullable = true)



[Row(height=170, name=u'kim, js', year=1)]

In [24]:
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType, TimestampType
r1=Row(name="js1",age=10)
r2=Row(name="js2",age=20)
_myRdd=spark.sparkContext.parallelize([r1,r2])
#Row 객체로 RDD를 생성할 수 있는 것이었다.

In [25]:
_myRdd.collect()

[Row(age=10, name='js1'), Row(age=20, name='js2')]

In [26]:
#createDataFrame함수에 (Rdd, schema)로 정의한 schema를 생성할 수 있다.
schema=StructType([
    StructField("age", IntegerType(), True),
    StructField("name", StringType(), True),
    #StructField("created", TimestampType(), True)
])
_myDf=spark.createDataFrame(_myRdd,schema)
_myDf.printSchema()
_myDf.show()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)

+---+----+
|age|name|
+---+----+
| 10| js1|
| 20| js2|
+---+----+



In [27]:
#위의 RDD->DF + Schema를 정리하면 아래와 같다.

from pyspark.sql.types import *
myRdd=spark.sparkContext.parallelize([(1, 'kim', 50.0), (2, 'lee', 60.0), (3, 'park', 70.0)])
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("height", DoubleType(), True)
])
_myDf = spark.createDataFrame(myRdd, schema)
_myDf.printSchema()
_myDf.show()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- height: double (nullable = true)

+---+----+------+
| id|name|height|
+---+----+------+
|  1| kim|  50.0|
|  2| lee|  60.0|
|  3|park|  70.0|
+---+----+------+



## Pandas

In [28]:
myDf.toPandas()

,year,name,height
0,1,"kim, js",170
1,1,"lee, sm",175
2,2,"lim, yg",180
3,2,lee,170


In [ ]:
#myDf.write.format('com.databricks.spark.csv').save(os.path.join('data','_myDf.csv')) #error 발생.

In [ ]:
#!doskey ls = dir #doskey ls란 이름으로 dir 기능을 구현

In [ ]:
%%writefile data\ds_twitter_seoul_3.json
{"contributors": null, "truncated": false, "text": "RT @soompi: #SEVENTEEN’s Mingyu, Jin Se Yeon, And Leeteuk To MC For 2016 Super Seoul Dream Concert \nhttps://t.co/1XRSaRBbE0 https://t.co/fi…", "is_quote_status": false, "in_reply_to_status_id": null, "id": 801657325836763136, "favorite_count": 0, "entities": {"symbols": [], "user_mentions": [{"id": 17659206, "indices": [3, 10], "id_str": "17659206", "screen_name": "soompi", "name": "Soompi"}], "hashtags": [{"indices": [12, 22], "text": "SEVENTEEN"}], "urls": [{"url": "https://t.co/1XRSaRBbE0", "indices": [100, 123], "expanded_url": "http://www.soompi.com/2016/11/20/seventeens-mingyu-jin-se-yeon-leeteuk-mc-dream-concert/", "display_url": "soompi.com/2016/11/20/sev…"}]}, "retweeted": false, "coordinates": null, "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>", "in_reply_to_screen_name": null, "in_reply_to_user_id": null, "retweet_count": 1487, "id_str": "801657325836763136", "favorited": false, "retweeted_status": {"contributors": null, "truncated": false, "text": "#SEVENTEEN’s Mingyu, Jin Se Yeon, And Leeteuk To MC For 2016 Super Seoul Dream Concert \nhttps://t.co/1XRSaRBbE0 https://t.co/fifXHpF8or", "is_quote_status": false, "in_reply_to_status_id": null, "id": 800593781586132993, "favorite_count": 1649, "entities": {"symbols": [], "user_mentions": [], "hashtags": [{"indices": [0, 10], "text": "SEVENTEEN"}], "urls": [{"url": "https://t.co/1XRSaRBbE0", "indices": [88, 111], "expanded_url": "http://www.soompi.com/2016/11/20/seventeens-mingyu-jin-se-yeon-leeteuk-mc-dream-concert/", "display_url": "soompi.com/2016/11/20/sev…"}], "media": [{"expanded_url": "https://twitter.com/soompi/status/800593781586132993/photo/1", "display_url": "pic.twitter.com/fifXHpF8or", "url": "https://t.co/fifXHpF8or", "media_url_https": "https://pbs.twimg.com/media/CxxHMk8UsAA4cUT.jpg", "id_str": "800593115165798400", "sizes": {"small": {"h": 382, "resize": "fit", "w": 680}, "large": {"h": 449, "resize": "fit", "w": 800}, "medium": {"h": 449, "resize": "fit", "w": 800}, "thumb": {"h": 150, "resize": "crop", "w": 150}}, "indices": [112, 135], "type": "photo", "id": 800593115165798400, "media_url": "http://pbs.twimg.com/media/CxxHMk8UsAA4cUT.jpg"}]}, "retweeted": false, "coordinates": null, "source": "<a href=\"https://about.twitter.com/products/tweetdeck\" rel=\"nofollow\">TweetDeck</a>", "in_reply_to_screen_name": null, "in_reply_to_user_id": null, "retweet_count": 1487, "id_str": "800593781586132993", "favorited": false, "user": {"follow_request_sent": false, "has_extended_profile": true, "profile_use_background_image": true, "default_profile_image": false, "id": 17659206, "profile_background_image_url_https": "https://pbs.twimg.com/profile_background_images/699864769/1cdde0a85f5c0a994ae1fb06d545a5ec.png", "verified": true, "translator_type": "none", "profile_text_color": "999999", "profile_image_url_https": "https://pbs.twimg.com/profile_images/792117259489583104/4khJk3zz_normal.jpg", "profile_sidebar_fill_color": "000000", "entities": {"url": {"urls": [{"url": "http://t.co/3evT80UlR9", "indices": [0, 22], "expanded_url": "http://www.soompi.com", "display_url": "soompi.com"}]}, "description": {"urls": []}}, "followers_count": 987867, "profile_sidebar_border_color": "000000", "id_str": "17659206", "profile_background_color": "1E1E1E", "listed_count": 3982, "is_translation_enabled": true, "utc_offset": -28800, "statuses_count": 80038, "description": "The original K-pop community. We take gifs, OTPs, and reporting on your bias' fashion choices seriously. But not rumors. Ain't nobody got time for that.", "friends_count": 3532, "location": "Worldwide", "profile_link_color": "31B6F4", "profile_image_url": "http://pbs.twimg.com/profile_images/792117259489583104/4khJk3zz_normal.jpg", "following": false, "geo_enabled": false, "profile_banner_url": "https://pbs.twimg.com/profile_banners/17659206/1478803767", "profile_background_image_url": "http://pbs.twimg.com/profile_background_images/699864769/1cdde0a85f5c0a994ae1fb06d545a5ec.png", "screen_name": "soompi", "lang": "en", "profile_background_tile": true, "favourites_count": 1493, "name": "Soompi", "notifications": false, "url": "http://t.co/3evT80UlR9", "created_at": "Wed Nov 26 20:48:27 +0000 2008", "contributors_enabled": false, "time_zone": "Pacific Time (US & Canada)", "protected": false, "default_profile": false, "is_translator": false}, "geo": null, "in_reply_to_user_id_str": null, "possibly_sensitive": false, "lang": "en", "created_at": "Mon Nov 21 06:56:46 +0000 2016", "in_reply_to_status_id_str": null, "place": null, "extended_entities": {"media": [{"expanded_url": "https://twitter.com/soompi/status/800593781586132993/photo/1", "display_url": "pic.twitter.com/fifXHpF8or", "url": "https://t.co/fifXHpF8or", "media_url_https": "https://pbs.twimg.com/media/CxxHMk8UsAA4cUT.jpg", "id_str": "800593115165798400", "sizes": {"small": {"h": 382, "resize": "fit", "w": 680}, "large": {"h": 449, "resize": "fit", "w": 800}, "medium": {"h": 449, "resize": "fit", "w": 800}, "thumb": {"h": 150, "resize": "crop", "w": 150}}, "indices": [112, 135], "type": "photo", "id": 800593115165798400, "media_url": "http://pbs.twimg.com/media/CxxHMk8UsAA4cUT.jpg"}]}, "metadata": {"iso_language_code": "en", "result_type": "recent"}}, "user": {"follow_request_sent": false, "has_extended_profile": false, "profile_use_background_image": true, "default_profile_image": true, "id": 791090169818521600, "profile_background_image_url_https": null, "verified": false, "translator_type": "none", "profile_text_color": "333333", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_6_normal.png", "profile_sidebar_fill_color": "DDEEF6", "entities": {"description": {"urls": []}}, "followers_count": 0, "profile_sidebar_border_color": "C0DEED", "id_str": "791090169818521600", "profile_background_color": "F5F8FA", "listed_count": 0, "is_translation_enabled": false, "utc_offset": null, "statuses_count": 96, "description": "", "friends_count": 7, "location": "", "profile_link_color": "1DA1F2", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_6_normal.png", "following": false, "geo_enabled": false, "profile_background_image_url": null, "screen_name": "enriquesanq", "lang": "es", "profile_background_tile": false, "favourites_count": 161, "name": "Enrique santos", "notifications": false, "url": null, "created_at": "Wed Oct 26 01:32:49 +0000 2016", "contributors_enabled": false, "time_zone": null, "protected": false, "default_profile": true, "is_translator": false}, "geo": null, "in_reply_to_user_id_str": null, "possibly_sensitive": false, "lang": "en", "created_at": "Thu Nov 24 05:22:55 +0000 2016", "in_reply_to_status_id_str": null, "place": null, "metadata": {"iso_language_code": "en", "result_type": "recent"}}


In [ ]:
import os
_jfname=os.path.join('data','ds_twitter_seoul_3.json')
with open(_jfname, 'rb') as f:
    data = f.readlines()
    
data

In [ ]:
data = map(lambda x: x.rstrip(), data)

In [ ]:
print "+".join(["A", "B", "C"]) #join 함수는 앞의 문자와 안의 인자를 결합

data_json_str = "[" + ','.join(data) + "]"

In [ ]:
len(data_json_str)

In [ ]:
import pandas as pd

data_df = pd.read_json(data_json_str)

In [ ]:
print data_df.count() #행의 개수  #전자는 column명이다.

In [ ]:
data_df['id'][:10]

In [58]:
bicycleRdd=spark.sparkContext.textFile(os.path.join('data','bicycle.csv'))

In [53]:
bicycleRdd.collect()

[u'\ufffd\ubfe9\ufffd\ufffd\ufffd\ufffd,\ufffd\ubfe9\ufffd\u01fc\ufffd',
 u'2018-01-01,4950',
 u'2018-01-02,7136',
 u'2018-01-03,7156',
 u'2018-01-04,7102',
 u'2018-01-05,7705',
 u'2018-01-06,5681',
 u'2018-01-07,5220',
 u'2018-01-08,6309',
 u'2018-01-09,5988',
 u'2018-01-10,4476',
 u'2018-01-11,4337',
 u'2018-01-12,4401',
 u'2018-01-13,3756',
 u'2018-01-14,4675',
 u'2018-01-15,6993',
 u'2018-01-16,7421',
 u'2018-01-17,6990',
 u'2018-01-18,7054',
 u'2018-01-19,8329',
 u'2018-01-20,6148',
 u'2018-01-21,5574',
 u'2018-01-22,4929',
 u'2018-01-23,4263',
 u'2018-01-24,3370',
 u'2018-01-25,3307',
 u'2018-01-26,3116',
 u'2018-01-27,2833',
 u'2018-01-28,3028',
 u'2018-01-29,4425',
 u'2018-01-30,3591',
 u'2018-01-31,4104',
 u'2018-02-01,5821',
 u'2018-02-02,6557',
 u'2018-02-03,3499',
 u'2018-02-04,2642',
 u'2018-02-05,4213',
 u'2018-02-06,4257',
 u'2018-02-07,5028',
 u'2018-02-08,6114',
 u'2018-02-09,6905',
 u'2018-02-10,4829',
 u'2018-02-11,3047',
 u'2018-02-12,5231',
 u'2018-02-13,6863',
 u'

In [60]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType
bicycleSchema=StructType([
    StructField("date", StringType(), True),
    StructField("count", IntegerType(), True)
])

In [59]:
bicycleRdd=bicycleRdd.map(lambda x:x.split(','))

In [61]:
bicycleRdd.collect()

[[u'2018-01-01', u'4950'],
 [u'2018-01-02', u'7136'],
 [u'2018-01-03', u'7156'],
 [u'2018-01-04', u'7102'],
 [u'2018-01-05', u'7705'],
 [u'2018-01-06', u'5681'],
 [u'2018-01-07', u'5220'],
 [u'2018-01-08', u'6309'],
 [u'2018-01-09', u'5988'],
 [u'2018-01-10', u'4476'],
 [u'2018-01-11', u'4337'],
 [u'2018-01-12', u'4401'],
 [u'2018-01-13', u'3756'],
 [u'2018-01-14', u'4675'],
 [u'2018-01-15', u'6993'],
 [u'2018-01-16', u'7421'],
 [u'2018-01-17', u'6990'],
 [u'2018-01-18', u'7054'],
 [u'2018-01-19', u'8329'],
 [u'2018-01-20', u'6148'],
 [u'2018-01-21', u'5574'],
 [u'2018-01-22', u'4929'],
 [u'2018-01-23', u'4263'],
 [u'2018-01-24', u'3370'],
 [u'2018-01-25', u'3307'],
 [u'2018-01-26', u'3116'],
 [u'2018-01-27', u'2833'],
 [u'2018-01-28', u'3028'],
 [u'2018-01-29', u'4425'],
 [u'2018-01-30', u'3591'],
 [u'2018-01-31', u'4104'],
 [u'2018-02-01', u'5821'],
 [u'2018-02-02', u'6557'],
 [u'2018-02-03', u'3499'],
 [u'2018-02-04', u'2642'],
 [u'2018-02-05', u'4213'],
 [u'2018-02-06', u'4257'],
 

In [62]:
bicycleDf=spark.createDataFrame(bicycleRdd, bicycleSchema)

In [63]:
bicycleDf.show()

Py4JJavaError: An error occurred while calling o497.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 29.0 failed 1 times, most recent failure: Lost task 0.0 in stage 29.0 (TID 226, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\G312\Downloads\spark-2.0.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 172, in main
  File "C:\Users\G312\Downloads\spark-2.0.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 167, in process
  File "C:\Users\G312\Downloads\spark-2.0.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\Users\G312\Downloads\spark-2.0.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\sql\session.py", line 505, in prepare
  File "C:\Users\G312\Downloads\spark-2.0.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\sql\types.py", line 1349, in _verify_type
